# 01 — Day 1 Walkthrough & Commands

This notebook is a reproducible log of what we did on **Day 1**: model probing/patching,
repository cleanup, Makefile fixes, and smoke validation. Run cells from the project root.

> Tip: replace `/workspace/jointdit` below if your path differs.

In [ ]:

# Project root used throughout this notebook
PROJECT_DIR = "/workspace/jointdit"
print("PROJECT_DIR =", PROJECT_DIR)



## A) Cross-attention coercion & UNet config alignment

We ensured **all cross-attention dims = 1024** and the UNet config matched.


In [ ]:

%%bash
set -euo pipefail
cd "${PROJECT_DIR}"
source .venv/bin/activate

# Coerce any inconsistent attn2 q/k/v weights to a single cross_attn dim
python scripts/aldm2/coerce_aldm2_cross_dim.py \  --target 1024 \  --model_dir assets/models/audioldm2/unet

# Force cross-attn everywhere in the UNet config (if needed)
python scripts/aldm2/force_cross_everywhere.py



## B) Strict component load (sanity)

We strictly loaded the ALDM2 components to catch size mismatches.
Expect to see "unused weights" warnings because the checkpoint contains extra keys.


In [ ]:

%%bash
set -euo pipefail
cd "${PROJECT_DIR}"
source .venv/bin/activate

# Strict UNet/VAE/cond-model load (keeps shapes honest)
python scripts/aldm2/smoke_aldm2_components.py --strict || true



## C) Repository tidy-up

We grouped scripts under topical subfolders: `scripts/{aldm2, clip, smoke, tools}`.


In [ ]:

%%bash
set -euo pipefail
cd "${PROJECT_DIR}"

# (If using git) sample moves — idempotent if already moved
# git mv scripts/coerce_aldm2_cross_dim.py scripts/aldm2/ || true
# git mv scripts/force_cross_everywhere.py   scripts/aldm2/ || true
# git mv scripts/smoke_svd_components.py    scripts/smoke/svd_smoke.py || true
# git mv scripts/smoke_aldm2_components.py  scripts/smoke/aldm2_smoke.py || true
# git commit -m "organize scripts into subfolders" || true

echo "Current layout:"
find scripts -maxdepth 2 -type f | sort



## D) Makefile fixes & targets

Use **tabs** (not spaces) for recipes. We added `smoke-aldm2` and `smoke-svd` targets.


In [ ]:

%%bash
set -euo pipefail
cd "${PROJECT_DIR}"

echo "Makefile excerpt (grep smoke targets):"
grep -nE 'svd_smoke\.py|aldm2_smoke\.py' Makefile || true



Example `Makefile` snippet (for reference):
```make
.PHONY: smoke-aldm2 smoke-svd smoke

smoke-aldm2:
	. .venv/bin/activate && python scripts/smoke/aldm2_smoke.py

smoke-svd:
	. .venv/bin/activate && python scripts/smoke/svd_smoke.py

smoke: smoke-aldm2 smoke-svd
```



## E) Run smoke tests with recommended CUDA allocator settings


In [ ]:

%%bash
set -euo pipefail
cd "${PROJECT_DIR}"
source .venv/bin/activate

export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:true,max_split_size_mb:128
echo "PYTORCH_CUDA_ALLOC_CONF=$PYTORCH_CUDA_ALLOC_CONF"

# Via Makefile (preferred):
make smoke-svd
make smoke-aldm2



## F) Expected smoke output cues

- Long list of **unused weights** → expected.
- `VAE scale = 4` (or 8 depending on model variant).
- Latent shape like `(1, 8, 80, 192)` and decoded spectrogram matching expected size
  `(1, 1, 320, 768)` for the current configs.
  
If these differ wildly or exceptions are raised, capture the logs and investigate.
